## Calculate bioclim variables using from terraclimate

In [1]:
import ee
ee.Initialize()
#import geemap
#Map = geemap.Map()
#from IPython.display import JSON
#import os

#### Load terraclimate collection

In [2]:
col = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')

In [3]:
# export polygon
roi = ee.Geometry.Polygon(ee.FeatureCollection("users/marcogirardello/boundboxes/boxglobal1").geometry().getInfo().get('coordinates')).bounds()

In [4]:
def mean_calc(img):
    tmp = img.select('tmmn').add(img.select('tmmx'))
    return tmp.divide(2).rename('tmean').copyProperties(img,['system:time_start'])

def bio2_calc(month):
    return tmax.filterMetadata('month','equals',month).first().\
    subtract(tmin.filterMetadata('month','equals',month)\
         .first())

In [5]:
# list of months
#months =ee.List(['%02d' % month for month in range(1,12+1)])
months = ee.List(list(range(1,12+1)))
# filters so to divide the year in quarters
q1 = ee.Filter.calendarRange(1,3,'month')
q2 = ee.Filter.calendarRange(4,6,'month')
q3 = ee.Filter.calendarRange(7,9,'month')
q4 = ee.Filter.calendarRange(10,12,'month')

In [6]:
# loop through years
#%env year = 2003
year = int(os.environ['year'])

NameError: name 'os' is not defined

In [7]:
prec = col.select('pr').filterDate(str(year)+'-01-01',str(year)+'-12-31')
tmin = col.select('tmmn').filterDate(str(year)+'-01-01',str(year)+'-12-31')
tmax = col.select('tmmx').filterDate(str(year)+'-01-01',str(year)+'-12-31')
tmean = col.filterDate(str(year)+'-01-01',str(year)+'-12-31').map(mean_calc)

NameError: name 'year' is not defined

In [44]:
# bio 1 (mean annual temperature)
bio1 = tmean.mean().multiply(0.1)
# bio 2 Mean Diurnal Range
bio2 = col.filterDate(str(year)+'-01-01',str(year)+'-12-31').\
map(lambda img: img.select('tmmx').subtract(img.select('tmmn'))).mean().rename('bio2')
# bio 4 (temperature seasonality)
bio4 = tmean.reduce(ee.Reducer.stdDev())
# bio 5 (max temperature of warmest month)
bio5 = tmax.max()
# bio 6 (min temperature of the coldest month)
bio6 = tmin.min()
# bio 7 (temperature annual range)
bio7 = bio5.subtract(bio6)
#bio 8 (mean temperature of wettest quarter) 
bio8 = ee.ImageCollection([prec.filter(q1).sum().addBands(tmean.filter(q1).mean()),
                    prec.filter(q2).sum().addBands(tmean.filter(q2).mean()),
                    prec.filter(q3).sum().addBands(tmean.filter(q3).mean()),
                    prec.filter(q4).sum().addBands(tmean.filter(q4).mean())]).reduce(ee.Reducer.max(2)).select('max1')
# bio 9 (mean temperature of driest quarter) 
bio9 = ee.ImageCollection([prec.filter(q1).sum().addBands(tmean.filter(q1).mean()),
                    prec.filter(q2).sum().addBands(tmean.filter(q2).mean()),
                    prec.filter(q3).sum().addBands(tmean.filter(q3).mean()),
                    prec.filter(q4).sum().addBands(tmean.filter(q4).mean())]).reduce(ee.Reducer.min(2)).select('min1')
# bio 10 (mean temperature of warmest quarter)
bio10 = ee.ImageCollection([tmean.filter(q1).mean(),tmean.filter(q2).mean(),tmean.filter(q3).mean(),tmean.filter(q4).mean()]).max()
# bio 11 (mean temperature of coldest quarter)
bio11 = ee.ImageCollection([tmean.filter(q1).mean(),tmean.filter(q2).mean(),tmean.filter(q3).mean(),tmean.filter(q4).mean()]).min()
# bio 12 (total annual precipitation)
bio12 = prec.sum().float()
# bio 13 (precipitation of wettest month)
bio13 = prec.max()
# bio 14 (precipitation of driest month)
bio14 = prec.min()
# bio 15 (precipitation seasonality (coef variation))
bio15 = prec.map(lambda image:image.add(1)).reduce(ee.Reducer.stdDev()).\
divide(prec.map(lambda image:image.add(1)).reduce(ee.Reducer.mean()))
bio15 = bio15.multiply(100)
# bio 16 (precipitation of wettest quarter)
bio16 = ee.ImageCollection([prec.filter(q1).sum(),prec.filter(q2).sum(),prec.filter(q3).sum(),prec.filter(q4).sum()]).max().float()
# bio 17 (precipitation of driest quarter)
bio17 = ee.ImageCollection([prec.filter(q1).sum(),prec.filter(q2).sum(),prec.filter(q3).sum(),prec.filter(q4).sum()]).min().float()
# bio 18 (precipitation of warmest quarter)
bio18 = ee.ImageCollection([tmean.filter(q1).mean().addBands(prec.filter(q1).sum()),
                          tmean.filter(q2).mean().addBands(prec.filter(q2).sum()),
                          tmean.filter(q3).mean().addBands(prec.filter(q3).sum()),
                          tmean.filter(q4).mean().addBands(prec.filter(q4).sum())]).reduce(ee.Reducer.max(2)).select('max1').float()
# bio 19 (precipitation of coldest quarter) 
bio19 = ee.ImageCollection([tmean.filter(q1).mean().addBands(prec.filter(q1).sum()),
                          tmean.filter(q2).mean().addBands(prec.filter(q2).sum()),
                          tmean.filter(q3).mean().addBands(prec.filter(q3).sum()),
                          tmean.filter(q4).mean().addBands(prec.filter(q4).sum())]).reduce(ee.Reducer.min(2)).select('min1').float()
# bio 3 (isothermality)
bio3 = bio2.divide(bio7).multiply(100)

In [49]:
finalres = ee.List([bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,bio10,bio11,bio12,bio13,bio14,bio15,
                   bio16,bio17,bio18,bio19])

In [66]:
# # bio19 for entire collection
# tmean = col.map(mean_calc)
# prec = col.select('pr')

# yearl = ee.List(list(range(1958,2000+1)))

# def bio_19glob(year):
#     temp_tmp = col.filter(ee.Filter.calendarRange(year,year,'year')).map(mean_calc)
#     prec_tmp = col.filter(ee.Filter.calendarRange(year,year,'year')).select('pr')
#     bio19 = ee.ImageCollection([temp_tmp.filter(q1).mean().addBands(prec_tmp.filter(q1).sum()),
#                           temp_tmp.filter(q2).mean().addBands(prec_tmp.filter(q2).sum()),
#                           temp_tmp.filter(q3).mean().addBands(prec_tmp.filter(q3).sum()),
#                           temp_tmp.filter(q4).mean().addBands(prec_tmp.filter(q4).sum())]).\
#                           reduce(ee.Reducer.min(2)).select('min1').float()
#     return bio19

# bio19glob = ee.ImageCollection(yearl.map(bio_19glob)).mean()

In [51]:
#%env biomin = 0
#env biomax = 19
#biomin = int(os.environ['biomin'])
#biomax = int(os.environ['biomax'])
biomin = 0
biomax = 19

In [50]:
#for i in range(biomin,biomax):
for i in range(biomin,biomax):
    print(i)
    tmp = ee.Image(finalres.get(i))
    task = ee.batch.Export.image.toCloudStorage(image = tmp,bucket='era5landbioclim',crs='EPSG:4326',
                                            fileFormat='GeoTIFF',
                                            maxPixels=1e13,scale=4638.312116386398,description='bio'+str(i+1)+'_'+str(year)+'_terraclimate',
                                            region = roi)
    task.start()

15
16
17
18


#### Export pet

In [12]:
pet = col.select('pet')

In [7]:
for year in range(2000,2020+1):
    print(year)
    pet_i = pet.filterDate(str(year)+'-01-01',str(year)+'-12-31')
    pet_year_i = pet_i.sum().toDouble()
    task = ee.batch.Export.image.toCloudStorage(image = pet_year_i,bucket='terraclimatepet',crs='EPSG:4326',
                                            fileFormat='GeoTIFF',
                                            maxPixels=1e13,scale=4638.312116386398,description='pet'+'_sum_'+str(year)+'_terraclimate',
                                            region = roi)
    task.start()

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


#### Export precipitation

In [18]:
pr = col.select('pr')

In [19]:
for year in range(2000,2020+1):
    print(year)
    pr_i = pr.filterDate(str(year)+'-01-01',str(year)+'-12-31')
    pr_year_i = pr_i.mean().toDouble()
    task = ee.batch.Export.image.toCloudStorage(image = pr_year_i,bucket='terraclimatepet',crs='EPSG:4326',
                                            fileFormat='GeoTIFF',
                                            maxPixels=1e13,scale=4638.312116386398,description='pr'+'_mean_'+str(year)+'_terraclimate',
                                            region = roi)
    task.start()

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
